In [8]:
!pip install unified-planning
# !pip install pydantic-ai
# !pip install requests
# !pip install "pydantic-ai-slim[mcp]"
from unified_planning.shortcuts import *


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [27]:
from unified_planning import Environment
env = Environment()
# peek at registered engine names
print(list(env.factory._engines.keys()))


['sequential_plan_validator', 'up_time_triggered_validator', 'sequential_simulator', 'up_bounded_types_remover', 'up_conditional_effects_remover', 'up_disjunctive_conditions_remover', 'up_state_invariants_remover', 'up_negative_conditions_remover', 'up_quantifiers_remover', 'up_usertype_fluents_remover', 'up_grounder', 'up_ma_disjunctive_conditions_remover', 'up_ma_conditional_effects_remover']


In [28]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import OneshotPlanner

# Parse the domain + problem
reader  = PDDLReader()
problem = reader.parse_problem('openrouter-domain.pddl', 'openrouter-problem.pddl')

# Instantiate the planner by name, not 'planner'
with OneshotPlanner(problem_kind=problem.kind) as planner:
    result = planner.solve(problem)
    if result.status.is_successful():
        print("Plan found:")
        for step in result.plan:
            print(step)
    else:
        print("No valid routing found.")


UPNoSuitableEngineAvailableException: No available OperationMode.ONESHOT_PLANNER engine

In [3]:
import cv2
import numpy as np

# Load YOLOv3 model with your local cfg and weights files
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Copied COCO class names for YOLOv3
classes = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog",
    "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite",
    "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle",
    "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange",
    "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant",
    "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard",
    "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock",
    "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# Get the output layer names for YOLOv3
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Initialize the Kalman Filter
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

# Function to update Kalman Filter with new measurements
def update_kalman_filter(kalman, measurement):
    kalman.correct(measurement)
    prediction = kalman.predict()
    return prediction

def detect_objects(frame):
    """
    Performs object detection on the given frame and returns bounding boxes and class IDs.
    Each bounding box is represented as (x, y, w, h).
    """
    # Create a blob from the image and forward it through the network
    blob = cv2.dnn.blobFromImage(frame, scalefactor=0.00392, size=(416, 416),
                                 mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    boxes = []
    confidences = []
    conf_threshold = 0.5
    nms_threshold = 0.4
    height, width = frame.shape[:2]

    # Loop over each of the detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # Only proceed if confidence is above threshold and detected object is a "car"
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))

    # Apply Non-Maximum Suppression to filter overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    detections = [boxes[i] for i in indices.flatten()] if len(indices) > 0 else []
    return detections

# --- Video processing using the detect_objects function ---
video_path = 'video1.mp4'
cap = cv2.VideoCapture(video_path)

# Setup output video writer with same resolution and FPS as input
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('task2.mp4', fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections = detect_objects(frame)
    for (x, y, w, h) in detections:
        # Measurement: center point of the detected object
        measurement = np.array([[np.float32(x + w / 2)],
                                [np.float32(y + h / 2)]])
        
        # Update Kalman Filter with the measurement
        prediction = update_kalman_filter(kalman, measurement)
        
        # Draw the predicted position
        cv2.circle(frame, (int(prediction[0]), int(prediction[1])), 4, (255, 0, 0), -1)
        # Draw the detected position and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    out.write(frame)
    
cap.release()
out.release()
cv2.destroyAllWindows()

print("Complete")


/var/folders/x7/dfxqs4mx3f3c9zdl5xrmj4k00000gn/T/ipykernel_80473/287301975.py:106: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.circle(frame, (int(prediction[0]), int(prediction[1])), 4, (255, 0, 0), -1)


Complete


In [27]:
!pip install --upgrade ultralytics
# !yt-dlp -f best --recode-video mp4 "https://www.youtube.com/watch?v=HYSdFHQDDcs&feature=youtu.be"

  Using cached ultralytics-8.3.96-py3-none-any.whl.metadata (35 kB)
Using cached ultralytics-8.3.96-py3-none-any.whl (949 kB)
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.43
    Uninstalling ultralytics-8.0.43:
      Successfully uninstalled ultralytics-8.0.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralyticsplus 0.0.28 requires ultralytics<8.0.44,>=8.0.43, but you have ultralytics 8.3.96 which is incompatible.


In [28]:
import cv2
import numpy as np
from collections import deque
from ultralytics import YOLO
import torch

# Load the model
model = YOLO('best.pt')

# Video capture setup (top-down view)
video_path = 'video1.mp4'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('task3.mp4', fourcc, fps, (frame_width, frame_height))

# Tracking parameters: For each track, we maintain a trajectory and a counted flag.
tracks = {}  # key: track_id, value: {'trajectory': deque, 'counted': bool}
next_track_id = 0
distance_threshold = 50     # maximum distance (in pixels) for matching centroids
trajectory_length = 10      # number of recent centroids to store per track

# Counters for vehicles moving left-to-right and right-to-left.
left_to_right_count = 0
right_to_left_count = 0
time_series = []  # To store (timestamp, left-to-right count, right-to-left count)

# For a top-down VisDrone model, the target vehicle class name(s) depend on the training.
# Here we assume the model’s label for vehicles is "car".
target_class = "car"
detection_threshold = 0.5  # Confidence threshold for detections

def update_tracks(detections, tracks, next_track_id):
    """
    Update the tracking dictionary with new detections using a simple nearest-neighbor matching.
    Each detection is a bounding box [x1, y1, x2, y2].
    """
    updated_tracks = {}
    for det in detections:
        x1, y1, x2, y2, centroid = det
        matched_id = None
        for track_id, data in tracks.items():
            last_point = data['trajectory'][-1]
            if np.linalg.norm(np.array(centroid) - np.array(last_point)) < distance_threshold:
                matched_id = track_id
                break
        if matched_id is not None:
            traj = tracks[matched_id]['trajectory']
            traj.append(centroid)
            if len(traj) > trajectory_length:
                traj.popleft()
            updated_tracks[matched_id] = {'trajectory': traj, 'counted': tracks[matched_id]['counted']}
        else:
            updated_tracks[next_track_id] = {'trajectory': deque([centroid], maxlen=trajectory_length), 'counted': False}
            next_track_id += 1
    return updated_tracks, next_track_id

def detect_vehicles(frame):
    """
    Runs the VisDrone-trained YOLOv8 model on the frame.
    Returns a list of detections for vehicles (e.g. "car").
    Each detection is a tuple: (x1, y1, x2, y2, (cx, cy))
    """
    results = model(frame)
    detections = []
    # Each result (if you pass one frame) contains a "boxes" attribute.
    for result in results:
        for box in result.boxes:
            conf = float(box.conf[0])
            class_id = int(box.cls[0])
            # Get class name from model.names dictionary
            label = model.names.get(class_id, "")
            if label.lower() == target_class and conf > detection_threshold:
                # box.xyxy is [x1, y1, x2, y2] in pixel coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cx = (x1 + x2) / 2.0
                cy = (y1 + y2) / 2.0
                detections.append((x1, y1, x2, y2, (cx, cy)))
    return detections

# Define vertical counting line at the middle (for left/right movement)
line_x = frame_width // 2

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect vehicles using the VisDrone-trained model
    detections = detect_vehicles(frame)
    for (x1, y1, x2, y2, centroid) in detections:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.circle(frame, (int(centroid[0]), int(centroid[1])), 4, (255, 0, 0), -1)

    # Update tracker with the current detections
    tracks, next_track_id = update_tracks(detections, tracks, next_track_id)

    # Determine if a tracked vehicle has crossed the vertical counting line.
    for track_id, data in tracks.items():
        traj = data['trajectory']
        if not data['counted'] and len(traj) >= 2:
            first_x = traj[0][0]
            last_x = traj[-1][0]
            # If the vehicle moves from left side to right side
            if first_x < line_x and last_x >= line_x:
                left_to_right_count += 1
                tracks[track_id]['counted'] = True
            # If the vehicle moves from right side to left side
            elif first_x > line_x and last_x <= line_x:
                right_to_left_count += 1
                tracks[track_id]['counted'] = True
        # Optionally draw the track ID near the latest centroid
        current_point = traj[-1]
        cv2.putText(frame, str(track_id), (int(current_point[0]), int(current_point[1])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Draw the vertical counting line
    cv2.line(frame, (line_x, 0), (line_x, frame_height), (0, 255, 255), 2)

    current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # seconds
    time_series.append((current_time, left_to_right_count, right_to_left_count))
    
    cv2.putText(frame, f"Left->Right: {left_to_right_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Right->Left: {right_to_left_count}", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    out.write(frame)
    cv2.imshow("Vehicle Counting (VisDrone Model)", frame)
    frame_idx += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# Normalize counts by video duration (vehicles per second)
total_duration = time_series[-1][0] if time_series else 0
if total_duration > 0:
    norm_left = left_to_right_count / total_duration
    norm_right = right_to_left_count / total_duration
    print("Final counts:")
    print("Left-to-Right:", left_to_right_count)
    print("Right-to-Left:", right_to_left_count)
    print("Normalized counts (vehicles per second):")
    print("Left-to-Right:", norm_left)
    print("Right-to-Left:", norm_right)



0: 384x640 13 cars, 1 van, 1 truck, 626.0ms
Speed: 19.6ms preprocess, 626.0ms inference, 17.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 van, 1 truck, 523.4ms
Speed: 2.9ms preprocess, 523.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 van, 1 truck, 963.6ms
Speed: 4.4ms preprocess, 963.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 675.5ms
Speed: 1.9ms preprocess, 675.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 van, 1 truck, 509.9ms
Speed: 1.7ms preprocess, 509.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 1022.4ms
Speed: 2.2ms preprocess, 1022.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 van, 1 truck, 714.7ms
Speed: 2.4ms preprocess, 714.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6

Final counts:
Left-to-Right: 28
Right-to-Left: 25
Normalized counts (vehicles per second):
Left-to-Right: 0.9259705811429949
Right-to-Left: 0.8267594474491027


: 